# PDF Loader

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from app.components.utils import load_env
env = load_env()

In [3]:
from app.components.media import PDFLoader
pdf_loader = PDFLoader()
# documents = pdf_loader.load_and_summarize(
    # "/workspaces/FoodRAG/assets/healthy_nutrition/10 healthier food options at hawker centres _ The Straits Times.pdf")

In [4]:
docs_text = pdf_loader.load(
    # "/workspaces/FoodRAG/assets/healthy_nutrition/101 Guide to Healthy Eating (English).pdf")
    "/workspaces/FoodRAG/assets/healthy_nutrition/10 healthier food options at hawker centres _ The Straits Times.pdf")
    # "/workspaces/FoodRAG/assets/healthy_nutrition/all-about-your-diet-booklet-for-prostate-cancer-patients.pdf")
    # "/workspaces/FoodRAG/assets/healthy_nutrition/Early chilhood nutrition from 1-2 years old.pdf")

In [5]:
len(docs_text)

2

In [6]:
print(docs_text[1].page_content)

6. Braised Duck Porridge


-----

PHOTO: WENSDELIGHT/INSTAGRAM

Who says that porridge is boring? While the generous amount of sauce often drizzled over this
dish can make it seem like a huge no-go when trying to eat healthily, you can always ask for less
gravy. What's left is a nutritious meal of lean duck meat with silky porridge. Yum.

7. Thunder Tea Rice


-----

PHOTO: SGSHUTTERBUG/INSTAGRAM

If you're finding it hard to incorporate two servings of vegetables into your meals, try this
delicious vegetable-topped rice dish.

The finely chopped selection of vegetables is accompanied with tofu, peanuts and ikan bilis, and
served with a thick, green soup made from healthy ingredients like green tea and coriander.

8. Mee Soto


-----

PHOTO: CAPRI.LICIOUS.EATS/INSTAGRAM

For lovers of spicy and savoury flavours, mee soto is a simple dish that packs a punch with
noodles made of flour, salt and egg. Ask for extra bean sprouts, so you get a good dose of veggies,
and don't finish the soup 

In [7]:
print("\n".join([doc.page_content for doc in docs_text]))

## THE STRAITS TIMES
 10 healthier food options at hawker centres

When eating yong tau foo, opt to have your ingredients boiled and served in soup. PHOTO: VERONICAPHUA/INSTAGRAM

UPDATED JUL 24, 2018, 05:20 PM

[(SHAPE) - Not all hawker food is unhealthy. These dishes are safe bets when you're watching](https://www.shape.com.sg/food/healthy-hawker-food-these-dishes-wont-make-you-feel-guilty-afterwards/)
[your diet. Plus, you can easily find them at your nearest hawker centre.](https://graphics.straitstimes.com/STI/STIMEDIA/Interactives/2018/07/healthy-eating/index.html)

It's normally easy enough to pack healthy meals and find clean-eating food joints for lunch. But
then, things get busy at the office and work piles up at your desk. Boom. Your healthy diet plans
are out the window because all you've got is 20 minutes for lunch, and the closest places to you
are the hawker centres.

Dang, you think. All they've got are fat-laden options like char kway teow or rojak. But take heart

-  

In [8]:
import pytesseract
import re


def detect_image_lang(img_path):
    osd = pytesseract.image_to_osd(img_path,output_type=pytesseract.Output.DICT)
    return osd['script']


detect_image_lang("/tmp/image_3.png")

TesseractError: (1, 'Error, cannot read input file /tmp/image_3.png: No such file or directory Error during processing.')

In [9]:
docs_reviser = pdf_loader.remove_unwanted_text(docs_text)

In [10]:
print("\n".join([doc.page_content for doc in docs_reviser]))

Here is the revised text:

THE STRAITS TIMES
10 Healthier Food Options at Hawker Centres

When eating yong tau foo, opt for boiled ingredients served in soup. PHOTO: VERONICAPHUA/INSTAGRAM

These dishes are safe bets when watching your diet. Plus, you can easily find them at your nearest hawker centre.

It's easy to pack healthy meals and find clean-eating food joints for lunch. But when work piles up at your desk, your healthy diet plans are out the window. Boom. You've got 20 minutes for lunch, and the closest places to you are the hawker centres.

Dang, you think. All they've got are fat-laden options like char kway teow or rojak. But take heart - not all local fare is full of sodium and calories. These yummy, affordable dishes are great for anyone trying to eat healthier.

Here are 10 relatively guilt-free hawker foods to choose from:

1. Sliced Fish Soup


This soup has a light flavour profile but is packed full of flavour and nutrition. Select sliced fish over its fried counterpa

In [ ]:
from app.components.media import ImageLoader
image_loader = ImageLoader()

In [ ]:
print(image_loader.load("/tmp/image_1.png"))

In [ ]:
import IPython.display as display
display.Image("/tmp/image_1.png")

# PDF Database

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from app.components.database import BaseDatabaseToolkit
pdf_db = BaseDatabaseToolkit(sql_database_url="sqlite:///database/usda.db")

/home/vscode/.local/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [ ]:
from langchain_core.messages import HumanMessage
pdf_db.invoke({"messages":[
  HumanMessage("What is the nutrition of banana"),]})

In [ ]:
pdf_db.get_tools()

In [ ]:
from app.components.database import BaseVectorDatabaseToolkit
from chromadb import HttpClient
vector_db = HttpClient(host="chroma")

In [ ]:
food_vector = BaseVectorDatabaseToolkit(vector_db)
food_vector.get_tools()

In [ ]:
from langchain_core.messages import HumanMessage
answ = food_vector.invoke({"messages":[
  HumanMessage("Can you save audio file?"),]})
for a in answ['messages']:
    print(a)

# Agent

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from langchain.globals import set_debug
set_debug(True)

In [ ]:
from app.components.agent import FoodDataAgent

In [ ]:
agent = FoodDataAgent(pdf_db, food_vector)

In [ ]:
agent.invoke({"messages":[
  HumanMessage("What is the nutrition of banana from sql?"),]})

In [2]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA
from langchain_openai import ChatOpenAI
from app.components.tools.hallucination import HallucinationGraderNVIDIA, HallucinationGraderOpenAI

print("Hallucination Grader")

documents = [
    {
        "title": "The history of the United States",
        "text": """The United States was founded in 1776 by a group of British colonists who wanted to break away from British rule. 
        The country was established as a constitutional republic, with a government that is divided into three branches: the executive, 
        legislative, and judicial branches. The United States has a long history of conflict and cooperation with other nations, 
        and has been involved in several major wars, including World War I and World War II. The country has also experienced periods 
        of internal conflict, such as the Civil War, which was fought over the issue
        """
    }
]

generation = "The United States was founded in 1777 by a group of British colonists who wanted to break away from British rule."
state = {"generation": generation, "documents": documents}
llm = ChatNVIDIA()
grader = HallucinationGraderNVIDIA(llm)
result = grader.invoke(state)
print(result)
llm2 = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
grader_openai = HallucinationGraderOpenAI(llm2)
result_openai = grader_openai.invoke(state)
print(result_openai)

Hallucination Grader
{'isHallucinate': 'no', 'reason': "The answer is grounded in the set of facts provided as the founding year was listed as 1776, which is one year earlier than the answer's claimed date."}
{'isHallucinate': 'yes', 'reason': 'The answer incorrectly states that the United States was founded in 1777, while the provided fact states it was founded in 1776.'}


In [3]:
from app.components.tools.answer import AnswerGraderNVIDIA, AnswerGraderOpenAI
print("Answer Grader")
question = "When was the United States founded?"
generation = "The United States was founded in 1777 by a group of British colonists who wanted to break away from British rule."
state = {"generation": generation, "question": question}
llm = ChatNVIDIA()
grader = AnswerGraderNVIDIA(llm)
result = grader.invoke(state)
print(result)
llm2 = ChatOpenAI(model="gpt-4o-mini", temperature=0.1)
grader_openai = AnswerGraderOpenAI(llm2)
result_openai = grader_openai.invoke(state)
print(result_openai)

Answer Grader
{'isUseful': False, 'reason': 'The answer does not provide the correct date for the foundation of the United States, as the country was actually founded in 1776, not 1777.'}
{'isUseful': True, 'reason': 'The answer provides the year the United States was founded, which directly addresses the question.'}
